# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [ ]:
!pip install -U --pre tensorflow=="2.*"
#!pip install tf_slim

Make sure you have `pycocotools` installed

In [ ]:
#!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


#if "models" in pathlib.Path.cwd().parts:
#  while "models" in pathlib.Path.cwd().parts:
#    os.chdir('..')
#elif not pathlib.Path('models').exists():
#  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [ ]:
#%%bash
#cd models/research/
#protoc object_detection/protos/*.proto --python_out=.###

In [ ]:
#%%bash 
#cd models/research
#pip install .

### Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import os
import pathlib
import imutils


from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [ ]:
#def load_model(model_name):
#  base_url = 'http://download.tensorflow.org/models/object_detection/'
#  model_file = model_name + '.tar.gz'
#  model_dir = tf.keras.utils.get_file(
#    fname=model_name, 
#    origin=base_url + model_file,
#    untar=True)
#   model_dir = pathlib.Path(model_dir)/"saved_model"
#    model_dir = "/home/user/TensorFlow/workspace/training_AS/exported-models/my_model/saved_model"
#    model = tf.saved_model.load(str(model_dir))
    
#    return model

#loaded_model = None

#def load_model(model_dir):
#    global loaded_model
    # load_model
#    model = tf.saved_model.load(str(model_dir))
#    return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/user/TensorFlow/workspace/training_AS/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

For the sake of simplicity we will test on 2 images:

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/home/user/TensorFlow/workspace/training_AS/images/test/')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

TEST_IMAGE_PATHS

# Detection

Load an object detection model:

In [ ]:
#model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model_dir = "/home/user/TensorFlow/workspace/training_AS/exported-models/my_model_201014/saved_model"
detection_model = tf.saved_model.load(str(model_dir))


Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [ ]:
detection_model.signatures["serving_default"].inputs

And returns several outputs:

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

In [ ]:
detection_model.signatures['serving_default'].structured_outputs

Add a wrapper function to call the model, and cleanup the outputs:

In [ ]:
def run_inference_for_single_image(model, image):
#    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                   for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
  
    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

Run it on each test image and show the results:

In [ ]:
def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

    display(Image.fromarray(image_np))

In [ ]:
# blue box represents positive, green box represent negative.

prediction_lst = []
error_imgs = []
count = 0

for image_path in TEST_IMAGE_PATHS:
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    image_np = imutils.resize(image_np, width=1024)
    # Actual detection.
    try:
        print(image_path)
        output_dict = run_inference_for_single_image(detection_model, image_np)
#        print(output_dict['detection_scores'][:3], output_dict['detection_classes'][:3])
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          min_score_thresh=0.7,
          line_thickness=8)
        output_dict["filename"] = image_path
        prediction_lst.append(output_dict)
        display(Image.fromarray(image_np))
    except Exception as ex:
        count += 1
        error_imgs.append(image_path)
        print(image_path)
        print(ex)
print(count)

## Confusion Matrix

In [ ]:
img_error_lst = [error_img.name for error_img in error_imgs]
img_error_lst

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import collections
csvpath = "/home/user/TensorFlow/workspace/training_AS/models/efficientdet_d4_coco17_tpu-32/test.csv"
csv = pd.read_csv(csvpath)
csv = csv.sort_values(by=["Filename", "BX"])
print(csv)
filename_lst = sorted(list(set(csv["Filename"].to_list())))
filename_lst

In [ ]:
import numpy as np
true_y_col = csv["label_SIJ"].to_list()
true_y = []
for i, file in enumerate(filename_lst):
    if file in img_error_lst:
        continue
    label = [true_y_col[2*i],true_y_col[2*i+1]]
    true_y += label
true_y = np.array(true_y)
true_y[true_y=="negative"] = 1
true_y[true_y=="positive"] = 2
true_y = list(map(int, true_y))
print(true_y, len(true_y))

In [ ]:
# get biggest prediction result
pred_y = []
score_dict = {}
for output_dict in prediction_lst[:]:
    detection_scores = output_dict["detection_scores"]
    detection_classes = output_dict["detection_classes"]
#    print(detection_classes[:6])
#    print(detection_scores[:6])
    detection_boxes = output_dict["detection_boxes"] #ymin, xmin, ymax, xmax
    filename = output_dict["filename"]
    position = ["right" if i[1] > 0.5 else "left" for i in detection_boxes]
#    print(position, position.index("right"), position.index("left"))
    pred_y += [detection_classes[position.index("left")],detection_classes[position.index("right")]]
    score_dict[filename] = (detection_scores[position.index("left")],detection_scores[position.index("right")])

print(pred_y, len(pred_y))

In [ ]:
# evaluate classification results
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

cm = confusion_matrix(true_y, pred_y, normalize='true')
print(classification_report(true_y, pred_y))

classes = [1,2]
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=0)
plt.yticks(tick_marks, classes)

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j], 3),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max()/2 else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()



In [ ]:
#save classification result to csv file
result_file = [i.name for i in score_dict.keys()]

result_df = pd.DataFrame(columns = col_list)
for index, row in csv.iterrows():
    filename = row["Filename"]
    if filename in result_file:
        result_df = result_df.append(row, sort=False)

#result_df.to_csv(os.path.join(train_path, "train.csv"))
print(result_df)


In [ ]:
import itertools

score_lst = [value for (key, value) in score_dict.items()]
#print(score_lst)
score_lst = list(itertools.chain(*score_lst))
print(score_lst, len(score_lst))

#pred_y = np.array(pred_y)
#pred_y[pred_y==1] = "negative"
#pred_y[pred_y==2] = "positive"
#pred_y = list(pred_y)

In [ ]:
result_df["classification_result"] = pred_y
result_df["score"] = score_lst
result_df

In [ ]:
result_df.to_csv("/home/user/TensorFlow/workspace/training_AS/exported-models/my_model_200925/result.csv")

# Shape Error

In [ ]:
import tensorflow as tf

with tf.compat.v1.Session() as sess:
    model_filename ='/home/user/TensorFlow/workspace/training_AS/exported-models/my_model_200925/saved_model/saved_model.pb'
    with tf.compat.v2.io.gfile.GFile(model_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='YOUR_LOG_LOCATION'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)